In [ ]:
promptlist = [
    
]

In [34]:
def generate(prompt, drawer, iterations):

  outdirectory = prompt.replace("8K", "").replace("3D", "").replace("HD", "").replace("#artstation", "").strip()
  torch.cuda.empty_cache()
  gc.collect()
  pixray.reset_settings()
  pixray.add_settings(prompts=prompt)
  pixray.add_settings(iterations=int(iterations))
  pixray.add_settings(batches=1)
  pixray.add_settings(learning_rate_drops=[100])
  pixray.add_settings(optimiser="DiffGrad")
  pixray.add_settings(clip_models="RN50,ViT-B/16,ViT-B/32")
  pixray.add_settings(quality="best")
  pixray.add_settings(size=[810,540])
  pixray.add_settings(display_clear="True")
  pixray.add_settings(make_video=False)
  pixray.add_settings(outdir=f"output/{outdirectory}")
  pixray.add_settings(drawer=drawer)

  settings = pixray.apply_settings()
  pixray.do_init(settings)
  pixray.do_run(settings)
  !cp "output/{outdirectory}/output.png" "output_finals/{outdirectory}.png"
  files.download(f"output_finals/{outdirectory}.png")


In [ ]:
def start_func():

    i = 0
    while i < len(promptlist):
        generate(promptlist[i], "vqgan", 480)
        i += 1

    !zip -r output_finals.zip output_finals/
    files.download("output_finals.zip")


In [ ]:
import os
if os.path.isfile("initiated"):
  import sys
  sys.path.append("pixray")
  import pixray
  import gc
  !mkdir models
  !mkdir output_finals
  import torch
  import base64
  from google.colab import files
  start_func()

else:
  !git clone --recursive --branch release https://github.com/pixray/pixray
  !pip install -r pixray/requirements.txt
  !pip install basicsr
  !pip install pybase64
  !pip uninstall -y tensorflow
  !git clone https://github.com/pixray/diffvg
  %cd diffvg
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  !pip freeze | grep torch
  !touch initiated
  print("initiated. ready to restart.")